In [ ]:
# !pip install -U yolo5face
# !pip install efficientnet_pytorch

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image, UnidentifiedImageError
import numpy as np
import os
import faiss
import matplotlib.pyplot as plt
import cv2
from yolo5face.get_model import get_model
from efficientnet_pytorch import EfficientNet


os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# 1.  유사도 모델

In [ ]:
# 예시 폴더 경로
folder_path = r"C:\Users\ben81\zflip_random2"

# EfficientNet 모델 로드 및 임베딩 벡터 추출 레이어 설정
base_model = models.efficientnet_b0(pretrained=True)
base_model.classifier[1] = nn.Identity()  # Remove the classification layer

# 이미지 전처리 함수
preprocess = transforms.Compose([
    transforms.Resize((256, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def get_image_embedding(image_path, model, device):
    try:
        # 이미지 로드 및 전처리
        img = Image.open(image_path).convert('RGB')
        img_tensor = preprocess(img).unsqueeze(0).to(device)

        # 임베딩 벡터 생성
        model.eval()
        with torch.no_grad():
            embedding_vector = model(img_tensor).cpu().numpy().flatten()
        return embedding_vector
    except UnidentifiedImageError:
        print(f"Cannot identify image file {image_path}. Skipping.")
        return None

def get_embeddings_from_folder(folder_path, model, device):
    embeddings = []
    image_paths = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(('jpg', 'jpeg', 'png')):
            image_path = os.path.join(folder_path, file_name)
            embedding = get_image_embedding(image_path, model, device)
            if embedding is not None:
                embeddings.append(embedding)
                image_paths.append(image_path)
    return np.array(embeddings), image_paths

def plot_image_groups(groups):
    for idx, group in enumerate(groups):
        if idx < 200:
            print("=================group {}=================".format(idx))
            plt.figure(figsize=(10,10))
            for i, image_path in enumerate(group):
                plt.subplot(1, len(group), i + 1)
                img = Image.open(image_path)
                plt.imshow(img)
                plt.axis('off')
            plt.show()


# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = base_model.to(device)

# 폴더 내 모든 이미지에 대한 임베딩 벡터 생성
embeddings, image_paths = get_embeddings_from_folder(folder_path, base_model, device)

# FAISS를 이용한 코사인 유사도 측정
index = faiss.IndexFlatIP(embeddings.shape[1])
faiss.normalize_L2(embeddings)
index.add(embeddings)

D, I = index.search(embeddings, k=len(embeddings))  # 모든 이미지에 대해 유사도 측정

# 유사도 0.775 이상인 그룹 생성
threshold = 0.775
groups = []
visited = set()

for i in range(len(embeddings)):
    if i in visited:
        continue
    group = [image_paths[i]]
    visited.add(i)
    for j in range(1, len(I[i])):
        if D[i][j] >= threshold and I[i][j] not in visited:
            group.append(image_paths[I[i][j]])
            visited.add(I[i][j])
    groups.append(group)

# 결과 출력
print(device)
plot_image_groups(groups)

# 2. blurring 모델

In [ ]:
# Define your model architecture
class MobileNetClassifier(nn.Module):
    def __init__(self):
        super(MobileNetClassifier, self).__init__()
        self.model = models.mobilenet_v2(pretrained=True)
        self.model.classifier[1] = nn.Linear(self.model.last_channel, 2)  # Assuming 2 classes: blur and sharp
    
    def forward(self, x):
        x = self.model(x)
        return x

# Initialize your model
blur_model = MobileNetClassifier()

# mobilenet_centered_softblurred_scene_laplacian
model_path = r"C:\Users\ben81\GitHub\pickle_archive\blur\models\mobilenet_centered_softblurred_scene_laplacian.pth"

# Load the saved blur_model state_dict
blur_model.load_state_dict(torch.load(model_path))

# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Device 설정 (GPU 또는 CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
blur_model.to(device)  # 모델을 GPU 또는 CPU로 전송

def predict_blur(blur_model, image_path):
    # 이미지 로드 및 전처리
    image = Image.open(image_path).convert('RGB')  # RGB로 변환
    image = transform(image).unsqueeze(0).to(device)  # 이미지를 GPU 또는 CPU로 전송

    # 모델 예측
    blur_model.eval()
    with torch.no_grad():
        output = blur_model(image)
        _, predicted = torch.max(output, 1)
        
    # 결과 출력
    classes = ['sharp','blur']
    predicted_class = classes[predicted.item()]
    if predicted_class == 'blur':
        # blur 사진 테스트용 -> 바로 사진 print
        # print(image_path)
        # img = cv2.imread(image_path,cv2.IMREAD_ANYCOLOR)
        # image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # plt.imshow(image_rgb)
        # plt.axis('off')  # 축 제거
        # plt.show()

        # blur된 사진 경로 return
        return image_path
    elif predicted_class == 'sharp':
        return None


path = r"C:\Users\ben81\GitHub\Pickle-DL\picklework\eyeclosing\zflip_camera"
blurred_files = []

for i in os.listdir(path):
    if i.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(path, i)
        try:
            blurred_path = predict_blur(blur_model, image_path)
            if blurred_path is not None:
                blurred_files.append(blurred_path)
        except:
            print('error')

# print(blurred_files)


In [ ]:
for image_path in blurred_files:
    image = cv2.imread(image_path)
    if image is None:
        print(f"{image_path}을(를) 로드하는 데 실패했습니다.")
        continue

    # BGR 이미지를 RGB로 변환
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 이미지 표시
    plt.figure(figsize=(6, 6))
    plt.imshow(image_rgb)
    plt.title(image_path)
    plt.axis('off')  # 축 숫자와 눈금 끄기
    plt.show()

# 3. Eye closing 모델

In [ ]:
import torch
import torch.nn as nn
import cv2
from torchvision import transforms
from PIL import Image
import os
from efficientnet_pytorch import EfficientNet

class EyeStateDetector:
    def __init__(self, images, model_path):
        self.images = images
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.yolo_model = self.load_yolo_model()
        self.efficientnet_model = self.load_efficientnet_model(model_path)
        self.class_names = ['ClosedFace', 'OpenFace']

    def load_yolo_model(self):
        model = get_model("yolov5n", device=self.device, min_face=24)
        return model

    def load_efficientnet_model(self, model_path):
        net = EfficientNet.from_pretrained('efficientnet-b3')
        num_features = net._fc.in_features
        net._fc = nn.Linear(num_features, 2)
        net.load_state_dict(torch.load(model_path, map_location=self.device))
        net = net.to(self.device)
        net.eval()

        return net

    def preprocess_image(self, image):
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)).convert('RGB')
        image = preprocess(image)
        image = image.unsqueeze(0)
        return image

    def face_detection(self, image):
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        boxes, key_points, scores = self.yolo_model(rgb_image, target_size=512)
        faces = []
        if len(boxes) > 0:
            for i, box in enumerate(boxes):
                x1, y1, x2, y2 = map(int, box)
                face_image = image[y1:y2, x1:x2]
                faces.append(face_image)
        return faces

    def eye_detection(self, face_image):
        image = self.preprocess_image(face_image).to(self.device)

        with torch.no_grad():
            outputs = self.efficientnet_model(image)
            _, preds = torch.max(outputs, 1)

        return self.class_names[preds.item()]

    def predict(self):
        results = {}

        for image_name, image in self.images.items():
            faces = self.face_detection(image)
            if not faces:
                results[image_name] = False
                continue

            eye_state = False
            for face in faces:
                eye_state = self.eye_detection(face)
                if eye_state == 'ClosedFace':
                    eye_state = True
                    break

            results[image_name] = eye_state

        return results

# Function to load all images from a folder
def load_images_from_folder(folder_path):
    images = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            if image is not None:
                images[filename] = image
    return images

# Example usage:
folder_path = r'C:\Users\ben81\zflip_random2'  # Replace with the actual folder path
model_path = r'C:\Users\ben81\zflip_random1'

# Load all images from the folder
images = load_images_from_folder(folder_path)

# Initialize the EyeStateDetector
detector = EyeStateDetector(images, model_path)

# Predict eye states
results = detector.predict()
print(results)


# 4. 지헌 Eye closing 모델

In [ ]:
from itertools import filterfalse
import torch
import torch.nn as nn
import cv2
from torchvision import transforms, models
from PIL import Image
import os
from ultralytics import YOLO

import logging
logging.basicConfig(level=logging.WARNING)


class EyeStateDetector:
    def __init__(self, images, model_path):
        self.images = images
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.yolo_model = self.load_yolo_model()
        self.efficientnet_model = self.load_efficientnet_model(model_path)
        self.class_names = ['ClosedFace', 'OpenFace']

    def load_yolo_model(self):
        model = YOLO("picklework/eyeclosing/yolov8n-face.pt")  # 모델 파일 경로 수정
        return model

    def load_efficientnet_model(self, model_path):
        net = models.efficientnet_b0(pretrained=True)
        in_features = net.classifier[1].in_features
        net.classifier[1] = nn.Linear(in_features, 2)
        net.load_state_dict(torch.load(model_path))

        net.to(self.device)
        net.eval()
        return net

    def preprocess_image(self, image):
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)).convert('RGB')
        image = preprocess(image)
        image = image.unsqueeze(0)
        return image

    def face_detection(self, image):
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = self.yolo_model.predict(source=rgb_image, conf=0.6, max_det=15, verbose=False)

        faces = []
        for result in results:
            for bbox in result.boxes.xyxy:
                x1, y1, x2, y2 = map(int, bbox)
                face_width, face_height = x2 - x1, y2 - y1

                # 최소 얼굴 크기 필터링
                if face_width >= 200 and face_height >= 200:
                    face_image = image[y1:y2, x1:x2]
                    faces.append(face_image)
        return faces

    def eye_detection(self, face_image):
        image = self.preprocess_image(face_image).to(self.device)

        with torch.no_grad():
            outputs = self.efficientnet_model(image)
            _, preds = torch.max(outputs, 1)

        return self.class_names[preds.item()]

    def predict(self):
        results = {}

        for image_name, image in self.images.items():
            faces = self.face_detection(image)
            if not faces:
                results[image_name] = False
                continue

            eye_state = False
            for face in faces:
                eye_state = self.eye_detection(face)
                if eye_state == 'ClosedFace':
                    eye_state = True
                    break

            results[image_name] = eye_state

        return results

# Function to load all images from a folder
def load_images_from_folder(folder_path):
    images = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            if image is not None:
                images[filename] = image
    return images

# Example usage:
folder_path = r"C:\Users\ben81\GitHub\Pickle-DL\picklework\eyeclosing\zflip_camera"  # Replace with the actual folder path
model_path = r"C:\Users\ben81\GitHub\Pickle-DL\picklework\eyeclosing\models\b0_3rd_rgbfinal_model.pth"

# Load all images from the folder
images = load_images_from_folder(folder_path)

# Initialize the EyeStateDetector
detector = EyeStateDetector(images, model_path)

# Predict eye states
results = detector.predict()
print(results)

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

for filename, result in results.items():
    file_path = os.path.join(folder_path, filename)
    
    if os.path.exists(file_path):
        # 이미지 읽기
        img = mpimg.imread(file_path)
        
        # 이미지 표시
        if result == True:
            plt.imshow(img)
            plt.title(f'{filename}: {result}')
            plt.axis('off')  # 축 숨기기
            plt.show()
    else:
        print(f'파일을 찾을 수 없습니다: {file_path}')

# 5. 흑백모델

In [ ]:
import torch
import torch.nn as nn
import cv2
from torchvision import transforms, models
from PIL import Image
import os
from ultralytics import YOLO

import logging
logging.basicConfig(level=logging.WARNING)


class EyeStateDetector:
    def __init__(self, images, model_path):
        self.images = images
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.yolo_model = self.load_yolo_model()
        self.efficientnet_model = self.load_efficientnet_model(model_path)
        self.class_names = ['ClosedFace', 'OpenFace']

    def load_yolo_model(self):
        model = YOLO("picklework/eyeclosing/yolov8n-face.pt")  # 모델 파일 경로 수정
        return model

    def load_efficientnet_model(self, model_path):
        weights = models.EfficientNet_B0_Weights.DEFAULT
        net = models.efficientnet_b0(weights=weights)
        net.features[0][0] = nn.Conv2d(1, net.features[0][0].out_channels, kernel_size=3, stride=2, padding=1, bias=False)
        in_features = net.classifier[1].in_features
        net.classifier[1] = nn.Linear(in_features, 2)
        net.load_state_dict(torch.load(model_path, map_location=self.device))

        net.to(self.device)
        net.eval()
        return net

    def preprocess_image(self, image):
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])  # 흑백 이미지 정규화
        ])
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))  # 이미지를 흑백으로 변환
        image = preprocess(image)
        image = image.unsqueeze(0)
        return image

    def face_detection(self, image):
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = self.yolo_model.predict(source=rgb_image, conf=0.65, max_det=15, verbose=False)

        faces = []
        for result in results:
            for bbox in result.boxes.xyxy:
                x1, y1, x2, y2 = map(int, bbox)
                face_width, face_height = x2 - x1, y2 - y1

                # 최소 얼굴 크기 필터링
                if face_width >= 200 and face_height >= 200:
                    face_image = image[y1:y2, x1:x2]
                    faces.append(face_image)
        return faces

    def eye_detection(self, face_image):
        image = self.preprocess_image(face_image).to(self.device)

        with torch.no_grad():
            outputs = self.efficientnet_model(image)
            _, preds = torch.max(outputs, 1)

        return self.class_names[preds.item()]

    def predict(self):
        results = {}

        for image_name, image in self.images.items():
            faces = self.face_detection(image)
            if not faces:
                results[image_name] = False
                continue

            eye_state = False
            for face in faces:
                eye_state = self.eye_detection(face)
                if eye_state == 'ClosedFace':
                    eye_state = True
                    break

            results[image_name] = eye_state

        return results

# Function to load all images from a folder
def load_images_from_folder(folder_path):
    images = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            if image is not None:
                images[filename] = image
    return images

# Example usage:
folder_path = r"C:\Users\ben81\GitHub\Pickle-DL\picklework\eyeclosing\zflip_camera"  # Replace with the actual folder path
model_path = r"C:\Users\ben81\GitHub\Pickle-DL\picklework\eyeclosing\models\b0_3rd_gray.pth"

# Load all images from the folder
images = load_images_from_folder(folder_path)

# Initialize the EyeStateDetector
detector = EyeStateDetector(images, model_path)

# Predict eye states
results = detector.predict()
print(results)


In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

for filename, result in results.items():
    file_path = os.path.join(folder_path, filename)
    
    if os.path.exists(file_path):
        # 이미지 읽기
        img = mpimg.imread(file_path)
        
        # 이미지 표시
        if result == True:
            plt.imshow(img)
            plt.title(f'{filename}: {result}')
            plt.axis('off')  # 축 숨기기
            plt.show()
    else:
        print(f'파일을 찾을 수 없습니다: {file_path}')